## Assignment: Automatic Cancer Diagnostic

Firstly, I will be importing the neccessary modules which are required to perform certain operations.Then, I will be loading the neccessary datasets using load_from_csv function.
This function will have one parameter, a file name. The function will read the CSV file and return a matrix (list of lists) in which a row in the file is a row in the matrix. 

In [1]:
import csv
import math
import operator
import statistics
import random

In [2]:
# Importing CSV to create a table-like custom objects from the items in CSV files.
def load_from_csv(filename):
    from csv import reader
    # Reading the csv file as a list of lists
    with open(filename, 'r') as read_obj:
    # Passing the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
    # Passing the reader object to list() to get a list of lists
        list_of_rows = list(csv_reader)
    for i in range(0, len(list_of_rows)): #coverting str to int
        for j in range(0, len(list_of_rows[i])):
            list_of_rows[i][j] = int(list_of_rows[i][j]) 
    return list_of_rows

We need to calculate Standard deviation. Standard deviation is a number that describes how spread out the values are. A low standard deviation means that most of the numbers are close to the mean (average) value. A high standard deviation means that the values are spread out over a wider range.
We need to calculate Standard deviation using get_standard_deviation function. In this function, it will have two parameters, a matrix and a column number. 
I will be using statistics module of Python which includes a function called statistics.stdev() which used to compute the standard deviation along the specified axis. 
It will return the standard deviation of the elements in the column number passed as a parameter. 


In [3]:
# Getting all the values for single column from data matrix
def get_standard_deviation(matrix, coln_num):
    coln_data = [row[coln_num] for row in matrix]
    std_dev = statistics.stdev(coln_data)
    return float("{:.3f}".format(std_dev))

Using the get_standard_deviation function above, we need to standardize the matrix to make sure that the data is internally consistent; that is, each data type has the same content and format. So, we need to use get_standardised_matrix function which returns a matrix containing the standardised version of the matrix passed as a parameter. Standardization can be done by subtracting the mean and dividing by the standard deviation for each value of each feature.

In [4]:
def get_standardised_matrix(matrix):
    data_std = []
    coln_len = len(matrix[0])
    for coln in range(coln_len):
        # Getting the standardize data of the column
        coln_data = [row[coln] for row in matrix]
        N = len(coln_data)
        avg = sum(coln_data) / N
        std_dev = get_standard_deviation(matrix, coln)
        data_std.append([(value - avg) / std_dev for value in coln_data])
        # Converting the standardize data in standardised matrix
    std_matrix = [
        [row[i] for row in data_std]
        for i in range(len(data_std[0]))]
    return std_matrix


We need to calculate the Euclidean distance between the two lists by using get_distance function. This function will have two parameters of out which both of them are lists. I will be using Euclidean distance formula to calculate the distance between two lists

In [5]:
def get_distance(list_a, list_b):
    if len(list_a) != len(list_b):
        return "Length of both list should be same"
    else:
        # Using the Euclidean distance formula between a list_a and a list_b
        eucl_dist = math.sqrt(sum([(num1 - num2) ** 2 for num1, num2 in zip(list_a, list_b)]))
        return eucl_dist

We need to find the nearest labels of each row in data that are closest to the learning data by using get distance function. We need to use get_k_nearest_labels function that will find the k rows of the matrix learning_data that are the closest to the list passed as a parameter. After finding these k rows, it will find and return the related rows in the matrix learning_data_labels. In get_k_nearest_labels function, I will be declaring an empty list which is used to store the distance values. Then, I will be using sort function to sort the distance values in ascending order. Using distance values, we can get the learning data labels for the number of values of k.

In [6]:
def get_k_nearest_labels(data, Learning_data, Learning_data_labels, k):
    dist_values = []   
    # Declaring empty list to store the euclidean distance between each row of data and learning data rows
    for i in range(len(Learning_data)):
        dist = get_distance(data, Learning_data[i])
        dist_values.append((Learning_data[i], dist, i))
        
     # Sorting the distances in ascending order
    dist_values.sort(key=operator.itemgetter(1))
    
    # Getting the k related rows in learning data labels
    closest_labels = [Learning_data_labels[dist_values[i][2]] for i in range(k)]
 
    return closest_labels    


After finding the nearest labels, we need to find the mode between the labels which can be done using get_mode function. This function will have one parameter, a matrix (list of lists). This matrix will have only one column which will be returned by get_k_nearest_labels. This function will return the mode of the numbers in this matrix. The mode of a sequence of numbers is the number with the highest frequency. Here i will be using collections, where we can import Counter function from collections. Counter holds the data in an unordered collection, just like hashtable objects. The elements here represent the keys and the count as values. It allows to count the items in an iterable list. 

Then, I am importing choice function from random which returns a random element from the sequence. If in the sequence, it has two numbers as highest frequency, then it returns a number random .

In [7]:
from collections import Counter
from random import choice


def get_mode(n_num):
    # Using list comprehension to convert a list of lists to a flat list to avoid unhashable lists
    n_num = [item for elem in n_num for item in elem]
    c = Counter(n_num)
    max_count = max(c.values())
    return choice([k for k in c if c[k] == max_count])

We need to build a Classification algorithm using the functions defined. In classification algorithm, We need to set a positive value for k. Then, for each row x in the matrix data. We need to find:
 1. Find the k rows in the matrix learning_data that are the closest to the row x in the matrix data according to the Euclidean distance.
 2. Given the rows found on Step 1, find the values of the same row numbers in the matrix learning_data_labels.
 3. Given the values found in Step 2, find the most common value. If two (or more) values have the same frequency, choose one of them at random.
 4. Set the row x of the matrix data_labels to the value found in Step 3.
 
We need to write a function called classify which returns a matrix described above. 
This function returns a matrix called the matrix data_labels which will contain an integer representing the label of the row x of the matrix data. A zero means patient x does not have Cancer and a one means the opposite.

In [8]:
def classify(std_data, std_learning_data, learning_data_labels, k):
    
    data_labels = []
    # Declaring an empty list to store the predicted values
    for i in std_data:
        data_labels.append(get_k_nearest_labels(i, std_learning_data, learning_data_labels, k))
    
    for j in range(0, len(data_labels)):
        data_labels[j] = get_mode(data_labels[j])
    
    return data_labels

After returning the matrix data_labels, we need to find the accuracy between data_labels and correct_data_labels. This can be done using a function called get_accuracy. This function will have two parameters. A matrix containing the data from the file correct_data_labels (Actual values), and a matrix containing the matrix data_labels (Predicted values). 
This function will calculate and return the percentage of accuracy. If both matrixes have exactly
the same values (in exactly the same row numbers) then the accuracy is of 100%. If only half of the
values of both tables match exactly in terms of value and row number, then the accuracy is of 50%,

In [9]:
def get_accuracy(actual, predicted):
    correct = 0
    N = len(actual)
    # Since our data_labels also contains list(not list of list) 
    # Converting correct_data_labels (actual) from list of lists data into list 
    actual = [j for i in actual for j in i]
    for i in range(N):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / N * 100

The last function is run_test. Here we need to create one matrix for each of these correct_data_labels, learning_data and learning_data_labels. It should standardise the matrix data and the matrix learning_data (using get_standardised_matrix). Then, it should run the algorithm (using classify) and calculate the accuracy (using get_acuracy) for a series of experiments. In each experiment you should run the algorithm (and calculate the accuracy) for different values of k (go from 3 to 15 in steps of 1), and show the results on the screen. 

In [10]:
def run_test():
    Data = load_from_csv('Data.csv')
    Learning_Data = load_from_csv('Learning_Data.csv')
    Learning_Data_Labels = load_from_csv('Learning_Data_Labels.csv')
    Correct_Data_Labels = load_from_csv('Correct_Data_Labels.csv')
    # Standardizing the matrix data and the matrix learning_data (using get_standardised_matrix).
    Data_Std = get_standardised_matrix(Data)
    Learning_Data_Std = get_standardised_matrix(Learning_Data)
    for k in range(3,16):
        data_labels = classify(Data_Std, Learning_Data_Std, Learning_Data_Labels, k)
        accuracy = get_accuracy(Correct_Data_Labels, data_labels)
        print(f"k={k}, Accuracy={accuracy:.3f}%")

In [11]:
run_test()

k=3, Accuracy=95.000%
k=4, Accuracy=95.000%
k=5, Accuracy=95.714%
k=6, Accuracy=94.286%
k=7, Accuracy=94.286%
k=8, Accuracy=94.286%
k=9, Accuracy=95.714%
k=10, Accuracy=95.714%
k=11, Accuracy=95.714%
k=12, Accuracy=95.714%
k=13, Accuracy=95.714%
k=14, Accuracy=95.714%
k=15, Accuracy=95.000%
